# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [1]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)

05/28/2021 13:49:39 - INFO - elasticsearch -   HEAD http://localhost:9200/aurelius [status:200 request:0.005s]
05/28/2021 13:49:39 - INFO - elasticsearch -   GET http://localhost:9200/aurelius [status:200 request:0.001s]
05/28/2021 13:49:39 - INFO - elasticsearch -   PUT http://localhost:9200/aurelius/_mapping [status:200 request:0.006s]
05/28/2021 13:49:39 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.001s]


Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [2]:
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.reader.farm import FARMReader

retriever = ElasticsearchRetriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

05/28/2021 13:52:46 - INFO - farm.utils -   Using device: CUDA 
05/28/2021 13:52:46 - INFO - farm.utils -   Number of GPUs: 1
05/28/2021 13:52:46 - INFO - farm.utils -   Distributed Training: False
05/28/2021 13:52:46 - INFO - farm.utils -   Automatic Mixed Precision: None
05/28/2021 13:52:54 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
05/28/2021 13:52:54 - INFO - farm.utils -   Using device: CUDA 
05/28/2021 13:52:54 - INFO - farm.utils -   Number of GPUs: 1
05/28/2021 13:52:54 - INFO - farm.utils -   Distributed Training: False
05/28/2021 13:52:54 - INFO - farm.utils -   Automatic Mixed Precision: None
05/28/2021 13:52:54 - INFO - farm.infer -   Got ya 19 parallel workers to do inference ...
05/28/2021 13:52:54 - INFO - farm.infer -    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
05/28/2021 13:52:54 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\  /|\  /w\ 

And now we initialize our ODQA pipeline.

In [3]:
from haystack.pipeline import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [4]:
qa.run(query='What did your grandfather teach?')

05/28/2021 13:54:38 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.045s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 87.66 Batches/s]


{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.1326475143432617,
 'answers': [{'answer': 'good morals and the government of my temper',
   'score': 5.544886589050293,
   'probability': 0.666658587300231,
   'context': 'From my grandfather Verus I learned good morals and the government of my temper.',
   'offset_start': 36,
   'offset_end': 79,
   'offset_start_in_doc': 36,
   'offset_end_in_doc': 79,
   'document_id': '5b09a8c3-4b4b-4b65-9a18-c7ce5e12a131',
   'meta': {'source': 'meditations'}},
  {'answer': 'self-government',
   'score': 1.4967514276504517,
   'probability': 0.5466375192351144,
   'context': 'From Maximus I learned self-government, and not to be led aside by anything; and cheerfulness in all circumstances, as well as in illness; and a just admixture in the moral character of sweetness and dignity, and to do what was set before me without complaining. I observed that everybody believed that he thought as he spoke, and that in all that he did he never h

In [5]:
qa.run(query='What is the Universe?',
       top_k_reader=3)

05/28/2021 13:56:04 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.006s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 77.55 Batches/s]


{'query': 'What is the Universe?',
 'no_ans_gap': 9.201206684112549,
 'answers': [{'answer': 'a well-arranged universe',
   'score': 12.65144157409668,
   'probability': 0.8294069733589583,
   'context': 'Either it is a well-arranged universe or a chaos huddled together, but still a universe. But can a certain order subsist in thee, and disorder in the All? And this too when all things are so separated and diffused and sympathetic.',
   'offset_start': 13,
   'offset_end': 37,
   'offset_start_in_doc': 13,
   'offset_end_in_doc': 37,
   'document_id': '149aa7b1-d5e3-416e-80cc-fa3355f8ab34',
   'meta': {'source': 'meditations'}},
  {'answer': 'the universe loves to make whatever is about to be',
   'score': 10.463760375976562,
   'probability': 0.7871732730339247,
   'context': '"The earth loves the shower"; and "the solemn aether loves": and the universe loves to make whatever is about to be. I say then to the universe, that I love as thou lovest. And is not this too said, that "this o

In [6]:
qa.run(query='What is art?',
       top_k_reader=3)

05/28/2021 13:59:16 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.006s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 77.54 Batches/s]


{'query': 'What is art?',
 'no_ans_gap': 4.066752910614014,
 'answers': [{'answer': 'the acts of life',
   'score': 6.919882774353027,
   'probability': 0.7037011827300649,
   'context': 'Let it make no difference to thee whether thou art cold or warm, if thou art doing thy duty; and whether thou art drowsy or satisfied with sleep; and whether ill-spoken of or praised; and whether dying or doing something else. For it is one of the acts of life, this act by which we die: it is sufficient then in this act also to do well what we have in hand.',
   'offset_start': 244,
   'offset_end': 260,
   'offset_start_in_doc': 244,
   'offset_end_in_doc': 260,
   'document_id': '1de6a444-ce8a-411a-a946-9e894b6e7e6e',
   'meta': {'source': 'meditations'}},
  {'answer': 'To be good',
   'score': 4.78495454788208,
   'probability': 0.6452259181632144,
   'context': 'What is thy art? To be good. And how is this accomplished well except by general principles, some about the nature of the universe, and o

Pretty cool stuff!